# Introducción, objetivos y contenido
====================================================================================================================================================================


El presente trabajo conforma la Parte 1 de 3 del trabajo final del sprint 14 para el curso de Data Analytics de la escuela TripleTen. Consiste en el análisis de resultados de un test A/B de una granja de sandías. Contenidos:

* Importación de librerías y carga de datos
* Preprocesamiento de datos
    * Verificación de tipo de datos
    * Verificación de duplicados
    * Verificación de nulos
* Análisis de datos (EDA)
    * Etapas de conversión (embudo)
    * Evaluación de distribución de eventos por usuario
    * Evaluar si hay usuarios presentes en ambas muestras
    * Distribución de eventos entre los días
    * Revisión de datos peculiares antes de iniciar la prueba A/B
* Evaluación de resultados
    * Resultados del test A/B
    * Comprobar la diferencia estadística entre las proporciones
* Conclusiones

# Importación y carga de datos
====================================================================================================================================================================

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from scipy.stats import mannwhitneyu

In [2]:
dfme = pd.read_csv('/Users/Juan/Documents/Academics/DataAnalytics/DA_S14_Final/DA_S14_Parte1_TestAB/ab_project_marketing_events_us.csv')
dfev = pd.read_csv('/Users/Juan/Documents/Academics/DataAnalytics/DA_S14_Final/DA_S14_Parte1_TestAB/final_ab_events_upd_us.csv')
dfnu = pd.read_csv('/Users/Juan/Documents/Academics/DataAnalytics/DA_S14_Final/DA_S14_Parte1_TestAB/final_ab_new_users_upd_us.csv')
dfpa = pd.read_csv('/Users/Juan/Documents/Academics/DataAnalytics/DA_S14_Final/DA_S14_Parte1_TestAB/final_ab_participants_upd_us.csv')

# Preprocesamiento de datos
====================================================================================================================================================================

## Dataset MARKETING EVENTS

In [3]:
dfme.info()
dfme

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes


,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11
5,Black Friday Ads Campaign,"EU, CIS, APAC, N.America",2020-11-26,2020-12-01
6,Chinese New Year Promo,APAC,2020-01-25,2020-02-07
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03
8,International Women's Day Promo,"EU, CIS, APAC",2020-03-08,2020-03-10
9,Victory Day CIS (May 9th) Event,CIS,2020-05-09,2020-05-11


In [4]:
# Filtrar eventos de marketing aplicables a la region "UE"
dfme = dfme[dfme['regions'].str.contains('EU')]

In [5]:
# Adecuación de tipos de datos
dfme['start_dt'] = pd.to_datetime(dfme['start_dt'])
dfme['finish_dt'] = pd.to_datetime(dfme['finish_dt'])
dfme.info()
dfme

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, 0 to 8
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   name       7 non-null      object        
 1   regions    7 non-null      object        
 2   start_dt   7 non-null      datetime64[ns]
 3   finish_dt  7 non-null      datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 280.0+ bytes


/var/folders/0_/sgcsk38j3dx9jkhktbsfmkgh0000gn/T/ipykernel_58824/948556076.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfme['start_dt'] = pd.to_datetime(dfme['start_dt'])
/var/folders/0_/sgcsk38j3dx9jkhktbsfmkgh0000gn/T/ipykernel_58824/948556076.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfme['finish_dt'] = pd.to_datetime(dfme['finish_dt'])


,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
5,Black Friday Ads Campaign,"EU, CIS, APAC, N.America",2020-11-26,2020-12-01
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03
8,International Women's Day Promo,"EU, CIS, APAC",2020-03-08,2020-03-10


In [6]:
dfme.duplicated().sum()

0

## Dataset EVENTS

In [7]:
dfev.info()
dfev.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     423761 non-null  object 
 1   event_dt    423761 non-null  object 
 2   event_name  423761 non-null  object 
 3   details     60314 non-null   float64
dtypes: float64(1), object(3)
memory usage: 12.9+ MB


,user_id,event_dt,event_name,details
53423,068149B12F12375F,2020-12-24 10:39:03,purchase,9.99
10511,D7B9AE3D57CBE4B6,2020-12-12 19:52:05,purchase,4.99


In [8]:
# Adecuación de campos
dfev['event_dt'] = pd.to_datetime(dfev['event_dt'])
dfev['event_date'] = dfev['event_dt'].dt.strftime('%Y-%m-%d')
dfev['revenue'] = pd.to_numeric(dfev['details'], errors='coerce')
dfev['revenue'].fillna(0, inplace=True)  # Por ejemplo, reemplazar NaN con 0

dfev = dfev[['user_id', 'event_dt', 'event_date', 'event_name', 'revenue']]
dfev.info()
dfev.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     423761 non-null  object        
 1   event_dt    423761 non-null  datetime64[ns]
 2   event_date  423761 non-null  object        
 3   event_name  423761 non-null  object        
 4   revenue     423761 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 16.2+ MB


,user_id,event_dt,event_date,event_name,revenue
185214,C06A4F46FAEBC52E,2020-12-19 01:54:34,2020-12-19,product_page,0.0
287535,15FAB67237ED0FF6,2020-12-14 07:24:37,2020-12-14,login,0.0


In [9]:
# Check de fechas entre 2020-12-07 y 2021-01-01
print(f'Fecha min: {dfev['event_date'].min()}')
print(f'Fecha max: {dfev['event_date'].max()}')


Fecha min: 2020-12-07
Fecha max: 2020-12-30


In [10]:
# Conteo de duplicados
dfev.duplicated().sum()

0

In [11]:
dfev['event_name'].unique()

array(['purchase', 'product_cart', 'product_page', 'login'], dtype=object)

## Dataset PARTICIPANTS

In [12]:
dfpa.info()
dfpa.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB


,user_id,group,ab_test
5620,175DBA167C2363B6,A,interface_eu_test
4175,1C224D9DB444296D,A,interface_eu_test


In [13]:
# Eliminar usuarios repetidos previo al recommend_ssytem_stesst
dfpa_test_recom = set(dfpa[dfpa['ab_test']=='recommender_system_test']['user_id'])
dfpa_test_inter = set(dfpa[dfpa['ab_test']=='interface_eu_test']['user_id'])
dfpa_users_duplicated = list(dfpa_test_recom.intersection(dfpa_test_inter))
dfpa = dfpa[~dfpa['user_id'].isin(dfpa_users_duplicated)]

# Filtramos los participantes del test A/B de interés
dfpa = dfpa[dfpa['ab_test']=='recommender_system_test']

dfpa.info()
dfpa.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 2788 entries, 0 to 3674
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  2788 non-null   object
 1   group    2788 non-null   object
 2   ab_test  2788 non-null   object
dtypes: object(3)
memory usage: 87.1+ KB


,user_id,group,ab_test
2011,FF0CF3A2B430E000,A,recommender_system_test
1653,82ADD32E61B5C939,B,recommender_system_test


In [14]:
dfpa.duplicated().sum()

0

## Dataset NEW USERS

In [15]:
dfnu.info()
dfnu.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     58703 non-null  object
 1   first_date  58703 non-null  object
 2   region      58703 non-null  object
 3   device      58703 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


,user_id,first_date,region,device
27442,3FA6A56C21E02725,2020-12-09,EU,Android
27972,FD80E5443BE70C72,2020-12-16,EU,PC


In [16]:
dfnu['first_date'] = pd.to_datetime(dfnu['first_date'])

# Filtrar nuevos usuarios por fecha: hasta el 21dic
dfnu = dfnu[dfnu['first_date']<='2020-12-21']

# Filtrar nuevos usuarios por region: "EU"
dfnu = dfnu[dfnu['region']=='EU']

dfnu.info()
dfnu.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 39466 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     39466 non-null  object        
 1   first_date  39466 non-null  datetime64[ns]
 2   region      39466 non-null  object        
 3   device      39466 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.5+ MB


,user_id,first_date,region,device
16547,D9C92EC5F9478381,2020-12-21,EU,Mac
58307,23099FEE162E1171,2020-12-20,EU,iPhone


In [17]:
dfnu.duplicated().sum()

0

## Creación de dataset de USUARIOS

In [18]:
df_users = pd.merge(dfpa, dfnu, on='user_id')

print(df_users.info())
df_users

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2594 entries, 0 to 2593
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     2594 non-null   object        
 1   group       2594 non-null   object        
 2   ab_test     2594 non-null   object        
 3   first_date  2594 non-null   datetime64[ns]
 4   region      2594 non-null   object        
 5   device      2594 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 121.7+ KB
None


,user_id,group,ab_test,first_date,region,device
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC
1,A7A3664BD6242119,A,recommender_system_test,2020-12-20,EU,iPhone
2,4FF2998A348C484F,A,recommender_system_test,2020-12-20,EU,Mac
3,7473E0943673C09E,A,recommender_system_test,2020-12-16,EU,iPhone
4,C46FE336D240A054,A,recommender_system_test,2020-12-17,EU,iPhone
...,...,...,...,...,...,...
2589,700E7D23A9A81AE3,B,recommender_system_test,2020-12-16,EU,Android
2590,7C5C12FA1B5AB710,A,recommender_system_test,2020-12-21,EU,Android
2591,91C3969B8A72B908,B,recommender_system_test,2020-12-09,EU,Android
2592,E26F13A65CEAC6EA,A,recommender_system_test,2020-12-17,EU,Mac


## Creación de dataset de ORDERS

In [21]:
df_orders = pd.merge(dfev, df_users, on='user_id', how='inner')
df_orders = df_orders[df_orders['region']=='EU']

print(df_orders.info())
df_orders.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16959 entries, 0 to 16958
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     16959 non-null  object        
 1   event_dt    16959 non-null  datetime64[ns]
 2   event_date  16959 non-null  object        
 3   event_name  16959 non-null  object        
 4   revenue     16959 non-null  float64       
 5   group       16959 non-null  object        
 6   ab_test     16959 non-null  object        
 7   first_date  16959 non-null  datetime64[ns]
 8   region      16959 non-null  object        
 9   device      16959 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(7)
memory usage: 1.3+ MB
None


,user_id,event_dt,event_date,event_name,revenue,group,ab_test,first_date,region,device
2849,E02E9EA00519B0CA,2020-12-24 15:06:53,2020-12-24,product_page,0.0,A,recommender_system_test,2020-12-15,EU,Mac
6898,C59B1E478A22FBE9,2020-12-07 17:57:15,2020-12-07,product_cart,0.0,A,recommender_system_test,2020-12-07,EU,iPhone
823,A3AD6954A1CBD2AF,2020-12-09 13:33:39,2020-12-09,login,0.0,A,recommender_system_test,2020-12-08,EU,PC
11682,34CC6C202D629117,2020-12-09 04:59:10,2020-12-09,login,0.0,A,recommender_system_test,2020-12-09,EU,Android
5653,A4FB6B61C861EB1F,2020-12-27 11:50:00,2020-12-27,login,0.0,A,recommender_system_test,2020-12-20,EU,PC
3996,77538B64EF289B42,2020-12-26 13:29:36,2020-12-26,login,0.0,A,recommender_system_test,2020-12-17,EU,Android
1528,A485880B6E55E212,2020-12-20 14:11:57,2020-12-20,product_cart,0.0,A,recommender_system_test,2020-12-13,EU,Android
3460,3175038D993E72DA,2020-12-20 11:56:14,2020-12-20,login,0.0,A,recommender_system_test,2020-12-16,EU,iPhone
8742,A9FD3F191C010525,2020-12-17 16:04:26,2020-12-17,login,0.0,A,recommender_system_test,2020-12-15,EU,Android
12464,950CD6AE83F84E7D,2020-12-18 11:37:47,2020-12-18,product_page,0.0,A,recommender_system_test,2020-12-14,EU,Android


In [22]:
df_orders['region'].unique()

array(['EU'], dtype=object)

## Preprocesamiento de datos - RESUMEN

* Tipos de datos: se modifican algunos campos que son necesarios visualizarlos como tipo "fecha"
* Valores nulos: no se observan valores nulos
* Valores duplicados: no se observan valores duplicados

# Análisis de datos
====================================================================================================================================================================

## Etapas de conversión (Embudo)

In [24]:
funnel = pd.pivot_table(df_orders, index='event_date', columns='event_name', values='user_id', aggfunc='count')

funnel = funnel.reset_index()
funnel

event_name,event_date,login,product_cart,product_page,purchase
0,2020-12-07,217.0,56.0,142.0,74.0
1,2020-12-08,189.0,52.0,117.0,50.0
2,2020-12-09,245.0,67.0,154.0,66.0
3,2020-12-10,202.0,60.0,112.0,54.0
4,2020-12-11,187.0,49.0,101.0,42.0
5,2020-12-12,203.0,54.0,115.0,51.0
6,2020-12-13,170.0,39.0,89.0,38.0
7,2020-12-14,434.0,135.0,260.0,119.0
8,2020-12-15,422.0,144.0,258.0,120.0
9,2020-12-16,450.0,151.0,285.0,137.0


## Cantidad de eventos por usuario

## Usuarios de ambas muestras

## Cantidad de eventos diarios (Serie de tiempo)

In [ ]:
plt.figure(figsize=(14, 3))
ax = sns.histplot(data=dfev, x='event_dt')

plt.title("Histograma de fecha y hora")
plt.xlabel("Hora")
plt.ylabel("Frecuencia")
plt.xticks(rotation=45)
plt.show()

## Cantidad de pedidos por usuario

In [ ]:
ordersByUsers = dfev.groupby(['user_id']).agg({'event_dt':'nunique'})
ordersByUsers.rename(columns={'event_dt': 'orders_amount'}, inplace=True)
plt.hist(ordersByUsers['orders_amount'])

## Ingresos por pedido

In [ ]:
ordersByUsers = dfev.groupby(['user_id']).agg({'revenue':'sum'})

plt.hist(ordersByUsers['revenue'])

# Evaluación de resultados del test A/B
====================================================================================================================================================================

# Conclusiones
====================================================================================================================================================================